In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard-of-oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [3]:
string_to_int ={ ch:i for i,ch in enumerate(chars)}
int_to_string ={ i:ch for i,ch in enumerate(chars)}
encode = lambda s:[string_to_int[c]for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data =torch.tensor(encode(text),dtype=torch.long)
print(data)

In [4]:
encoded_string=encode('hello')
print(decode(encoded_string))

hello


In [5]:
data =torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([91, 48, 65, 62,  1, 44, 75, 72, 67, 62, 60, 77,  1, 35, 78, 77, 62, 71,
        59, 62, 75, 64,  1, 62, 30, 72, 72, 68,  1, 72, 63,  1, 32, 72, 75, 72,
        77, 65, 82,  1, 58, 71, 61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,
         1, 66, 71,  1, 43, 83,  0,  1,  1,  1,  1,  0, 48, 65, 66, 76,  1, 62,
        59, 72, 72, 68,  1, 66, 76,  1, 63, 72, 75,  1, 77, 65, 62,  1, 78, 76,
        62,  1, 72, 63,  1, 58, 71, 82, 72, 71])


In [6]:
n=int(.8*len(data))
train_data=data[:n]
test_data=data[n:]

In [7]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[82, 14,  0,  3, 37, 77,  8, 76],
        [61,  1, 77, 65, 62,  1, 80, 58],
        [60, 72, 71, 76, 77, 75, 78, 60],
        [80, 72,  1, 75, 62, 58, 76, 72]], device='cuda:0')
targets:
tensor([[14,  0,  3, 37, 77,  8, 76,  1],
        [ 1, 77, 65, 62,  1, 80, 58, 71],
        [72, 71, 76, 77, 75, 78, 60, 77],
        [72,  1, 75, 62, 58, 76, 72, 71]], device='cuda:0')


In [8]:

x= train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print('when input is', context ,'target is ',target)


when input is tensor([91]) target is  tensor(48)
when input is tensor([91, 48]) target is  tensor(65)
when input is tensor([91, 48, 65]) target is  tensor(62)
when input is tensor([91, 48, 65, 62]) target is  tensor(1)
when input is tensor([91, 48, 65, 62,  1]) target is  tensor(44)
when input is tensor([91, 48, 65, 62,  1, 44]) target is  tensor(75)
when input is tensor([91, 48, 65, 62,  1, 44, 75]) target is  tensor(72)
when input is tensor([91, 48, 65, 62,  1, 44, 75, 72]) target is  tensor(67)


In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


2jQ-hAOd'jr&]%—z’#Uw6
 jV/Y1KIfDiLWiW—84W”‘xZ1MVGM;v!kYyZz“r,xVrpyo﻿G#oCXRkiHGu$tAOCeP‘)Cym)n[8OUSM_W
cwW%:8$:PB9™q(1&Q‘O!P$%DYW*QL‘6dG.bN38‘17JZZgUv0Cz84﻿[KKf]x]:rN•RIsnPMC]JOlxePBS0d6"is[xyX•qGRBs9$UUk FQH4j‘#’‘﻿L*Bkn EKqQ﻿b-XRguDZc—kx]nA’keFyl"[)NvNQ]™TIDuQ*b"3h’sCJXR(d%GeuPL.p[Y.i;JuD
nRmd!”xV5”,K"noCl﻿,17V4
En. g•XtRn]?VhNo_0GQLi!jJvaM/*M;mMf—0UUGun")Nri;—szqWT”OEFNr?j’HvJ3.2E
0OFwG%t $7ZqQO,4"Jo’A0dqWK﻿EYXk(NrOFLWNrVG'PBqQC]“$z]Yz3#)Fr﻿:-;/ZqWw6T7Cl)#QM_i8bXGH&liuy(_“24
);$Lk/q)C-%& ,lJz;/


In [11]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.942, val loss: 4.944
step: 250, train loss: 4.879, val loss: 4.888
step: 500, train loss: 4.813, val loss: 4.844
step: 750, train loss: 4.754, val loss: 4.766
step: 1000, train loss: 4.686, val loss: 4.716
step: 1250, train loss: 4.650, val loss: 4.660
step: 1500, train loss: 4.557, val loss: 4.571
step: 1750, train loss: 4.507, val loss: 4.543
step: 2000, train loss: 4.441, val loss: 4.463
step: 2250, train loss: 4.363, val loss: 4.419
step: 2500, train loss: 4.330, val loss: 4.375
step: 2750, train loss: 4.272, val loss: 4.319
step: 3000, train loss: 4.217, val loss: 4.273
step: 3250, train loss: 4.173, val loss: 4.228
step: 3500, train loss: 4.142, val loss: 4.153
step: 3750, train loss: 4.054, val loss: 4.106
step: 4000, train loss: 4.008, val loss: 4.077
step: 4250, train loss: 3.982, val loss: 4.037
step: 4500, train loss: 3.932, val loss: 3.979
step: 4750, train loss: 3.880, val loss: 3.947
step: 5000, train loss: 3.833, val loss: 3.907
step: 5250, train l